<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Source of the data:
https://data.world/lpetrocelli/lendingclub-loan-data-2017-q-1/workspace/project-summary?agentid=lpetrocelli&datasetid=lendingclub-loan-data-2017-q-1

Source:
https://www.lendingclub.com/info/download-data.action

LendingClub is a peer-to-peer lending company with its headquarters in San Francisco, California, began by operating as an online consumer-lending platform that enables borrowers to obtain a loan that's funded by individuals and insititutions.  They issue loans for personal - debt consolidation, medical bills or home improvement.   

When a borrower applies for a loan, LendingClub looking at the person's income, credit history, employment.  There are two types of decisions to be paid by LC - accept the loan or reject the loan.  In accepting the loan, there is a chance that the borrower will repay the loan or it it will charged off.  


Problem Statement:
The goal is to understand what factors affect a loan in default.  Using some of the features in the dataset, we will determine if someone will pay or default on their loan.

In [13]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [14]:
loan = pd.read_csv('./LoanStats/accepted_2007_to_2018Q4.csv', low_memory=False)


In [16]:
x = loan[['funded_amnt', 'emp_length', 'annual_inc', 'loan_status', 
         'loan_amnt', 'home_ownership', 'int_rate','addr_state', 'grade', 
          'term', 'installment', 'sub_grade', 'total_pymnt','issue_d', 
          'funded_amnt_inv','purpose', 'dti', 'zip_code']]
loan2 = x.copy()

In [17]:
# Release the memory of loan since we don't need it anymore
del loan

In [18]:
loan2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   funded_amnt      float64
 1   emp_length       object 
 2   annual_inc       float64
 3   loan_status      object 
 4   loan_amnt        float64
 5   home_ownership   object 
 6   int_rate         float64
 7   addr_state       object 
 8   grade            object 
 9   term             object 
 10  installment      float64
 11  sub_grade        object 
 12  total_pymnt      float64
 13  issue_d          object 
 14  funded_amnt_inv  float64
 15  purpose          object 
 16  dti              float64
 17  zip_code         object 
dtypes: float64(8), object(10)
memory usage: 310.5+ MB


In [19]:
loan2.to_csv('./LoanStats/loans_with_date.csv')

In [20]:
loan2.shape

(2260701, 18)

In [19]:
loan2.isnull().sum()

funded_amnt           33
emp_length        146940
annual_inc            37
loan_status           33
home_ownership        33
int_rate              33
addr_state            33
grade                 33
issue_d               33
purpose               33
dti                 1744
term                  33
zip_code              34
dtype: int64

In [18]:
loan2.dropna()

,funded_amnt,emp_length,annual_inc,loan_status,home_ownership,int_rate,addr_state,grade,issue_d,purpose,dti,term,zip_code
0,3600.0,10+ years,55000.0,Fully Paid,MORTGAGE,13.99,PA,C,Dec-2015,debt_consolidation,5.91,36 months,190xx
1,24700.0,10+ years,65000.0,Fully Paid,MORTGAGE,11.99,SD,C,Dec-2015,small_business,16.06,36 months,577xx
2,20000.0,10+ years,63000.0,Fully Paid,MORTGAGE,10.78,IL,B,Dec-2015,home_improvement,10.78,60 months,605xx
3,35000.0,10+ years,110000.0,Current,MORTGAGE,14.85,NJ,C,Dec-2015,debt_consolidation,17.06,60 months,076xx
4,10400.0,3 years,104433.0,Fully Paid,MORTGAGE,22.45,PA,F,Dec-2015,major_purchase,25.37,60 months,174xx
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260694,24000.0,7 years,95000.0,Current,MORTGAGE,12.79,AL,C,Oct-2016,home_improvement,19.61,60 months,356xx
2260695,24000.0,10+ years,108000.0,Current,MORTGAGE,10.49,UT,B,Oct-2016,debt_consolidation,34.94,60 months,840xx
2260696,40000.0,9 years,227000.0,Current,MORTGAGE,10.49,CA,B,Oct-2016,debt_consolidation,12.75,60 months,907xx
2260697,24000.0,6 years,110000.0,Charged Off,RENT,14.49,FL,C,Oct-2016,debt_consolidation,18.30,60 months,334xx


In [20]:
def employment_length(text):
    text = text.lower()
    
    if text == 'nan':
        text = '0'
    elif text == 'n/a':
        text = '0'
    elif text == '< 1 year':
        text = '0.5'
    elif text == '1 year':
        text = '1'
    elif text == '2 years':
        text = '2'
    elif text == '3 years':
        text = '3'
    elif text == '4 years':
        text = '4'
    elif text == '5 years':
        text = '5'
    elif text == '6 years':
        text = '6'
    elif text == '7 years':
        text = '7'
    elif text == '8 years':
        text = '8'
    elif text == '9 years':
        text = '9'
    elif text == '10+ years':
        text = '10'
            
    return text

In [21]:
loan2['emp_length'] = loan2['emp_length'].astype('str')
loan2['emp_length'] = [employment_length(i) for i  in loan2['emp_length']]
loan2['emp_length'] = loan2['emp_length'].astype('float')
